# Build the Classifier

In [1]:
import numpy as np
import tflearn

W0807 02:56:59.637835 139782408525632 deprecation_wrapper.py:119] From /home/wangyu/miniconda3/lib/python3.7/site-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0807 02:56:59.639188 139782408525632 deprecation_wrapper.py:119] From /home/wangyu/miniconda3/lib/python3.7/site-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0807 02:56:59.644663 139782408525632 deprecation_wrapper.py:119] From /home/wangyu/miniconda3/lib/python3.7/site-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0807 02:56:59.648335 139782408525632 deprecation_wrapper.py:119] From /home/wangyu/miniconda3/lib/python3.7/site-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0807 02:56:59.653948 139782408525

## Loading Data

In [2]:
# Download the Titanic dataset
from tflearn.datasets import titanic
titanic.download_dataset('titanic_dataset.csv')

# Load CSV file, indicate that the first column represents labels
from tflearn.data_utils import load_csv
data, labels = load_csv('titanic_dataset.csv', target_column=0, categorical_labels=True, n_classes=2)

## Preprocessing Data

In [3]:
# Preprocessing function
def preprocess(data, columns_to_ignore):
    # Sort by descending id and delete columns
    for id in sorted(columns_to_ignore, reverse=True):
        data = [np.delete(r, id) for r in data]
    for i in range(len(data)):
        # Converting 'sex' field to float (id is 1 after removing labels column)
        data[i][1] = 1. if data[i][1] == 'female' else 0.
    return np.array(data, dtype=np.float32)

# Ignore 'name' and 'ticket' columns (id 1 & 6 of data array)
to_ignore=[1, 6]

# Preprocess data
data_removed = preprocess(data, to_ignore)

## Build a Deep Neural Network

In [8]:
# Build neural network
net = tflearn.input_data(shape=[None, 6])
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net)

<class 'tensorflow.python.framework.ops.Tensor'>


## Training

In [5]:
# Define model
model = tflearn.DNN(net)
# Start training (apply gradient descent algorithm)
model.fit(data_removed, labels, n_epoch=10, batch_size=16, show_metric=True)

Training Step: 819  | total loss: 0.49297 | time: 0.230s
| Adam | epoch: 010 | loss: 0.49297 - acc: 0.7869 -- iter: 1296/1309
Training Step: 820  | total loss: 0.48781 | time: 0.233s
| Adam | epoch: 010 | loss: 0.48781 - acc: 0.7895 -- iter: 1309/1309
--


## Try the model

In [6]:
# Let's create some data for DiCaprio and Winslet
dicaprio = [3, 'Jack Dawson', 'male', 19, 0, 0, 'N/A', 5.0000]
winslet = [1, 'Rose DeWitt Bukater', 'female', 17, 1, 2, 'N/A', 100.0000]
# Preprocess data
dicaprio, winslet = preprocess([dicaprio, winslet], to_ignore)
# Predict surviving chances (class 1 results)
pred = model.predict([dicaprio, winslet])
print("DiCaprio Surviving Rate:", pred[0][1])
print("Winslet Surviving Rate:", pred[1][1])

DiCaprio Surviving Rate: 0.14488289
Winslet Surviving Rate: 0.87632865


---